# <span style='color:Green'> Computer Vision Application for Autonomous Driving Application: </span>
# <span style='color:Blue'> Live Mapping of Traffic into 2D Navigation Map using YOLO_V3 </span>

In [1]:
import cv2
import numpy as np
import math

In [2]:
# Load Yolo model files
yolo_weight = "yolov3.weights"
yolo_config = "yolov3.cfg"
coco_labels = "coco.names"
net = cv2.dnn.readNet(yolo_weight, yolo_config)
 
# Load coco object names file
classes = []
with open(coco_labels, "r") as f:
    classes = [line.strip() for line in f.readlines()]

In [3]:
# Find names of all layers
layer_names = net.getLayerNames()
# print(layer_names)

In [4]:
# Find names of three output layers
output_layers = [layer_names[i - 1] for i in net.getUnconnectedOutLayers()]
# print(output_layers)

In [8]:
# Generating random colors assuming not more than 40 objects will be detected in a frame
colors = np.random.uniform(0, 255, size=(40, 3)) #len(classes)
# print(colors)

# <span style='color:Green'> Using video file - Dash Cam Traffic VIdeo </span>

In [6]:
# read video from file
cap = cv2.VideoCapture('trim16.mp4')

while True:
    read_ok, img = cap.read()
 
    height, width, channels = img.shape
    
    # Detecting objects
    blob = cv2.dnn.blobFromImage(img, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
    net.setInput(blob)
    outs = net.forward(output_layers)
 
    # Showing informations on the screen
    class_ids = []
    confidences = []
    boxes = []
    cars = []
    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.5:
                # Object detected
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                # Mapping into 2D
                yc = int(10*math.tan((1 - detection[1] - 0.5*detection[3]) *math.pi/2 + 0.83))
                # scaling factor* is not effective enough. so adding theta bias
                xc = int((center_x*400/width - 190)*(yc+15)/280 +200) #int(detection[0] * yc)
                xc2 = xc + 16
                yc2 = yc + 16
                w = int(detection[2] * width)
                h = int(detection[3] * height)
                # Rectangle coordinates
                x = int(center_x - w / 2)
                y = int(center_y - h / 2)
                boxes.append([x, y, w, h])
                cars.append([xc, yc, xc2, yc2])
                confidences.append(float(confidence))
                class_ids.append(class_id)
 
    indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)
 
    font = cv2.FONT_HERSHEY_DUPLEX
    
    img_1 = np.zeros([650,400,3],dtype=np.uint8)
    img_1.fill(255)
    cv2.rectangle(img_1, (190, 0), (210, 800), (255,220,255), -1)
    cv2.rectangle(img_1, (210, 0), (230, 800), (255,255,220), -1)
    cv2.rectangle(img_1, (230, 0), (250, 800), (255,220,255), -1)
    cv2.rectangle(img_1, (150, 0), (170, 800), (255,220,255), -1)
    cv2.rectangle(img_1, (170, 0), (190, 800), (255,255,220), -1)
    cv2.rectangle(img_1, (192, 0), (208, 16), (0,255,0), 2)
    cv2.putText(img_1, 'V', (192, 32), font, 0.8, (0,255,0), 2)
    
    for i in range(len(boxes)):
        if i in indexes:
            x, y, w, h = boxes[i]
            xc, yc, xc2, yc2 = cars[i]
            label = str(classes[class_ids[i]])
            confidence_label = int(confidences[i] * 100)
            color = colors[i]
            cv2.rectangle(img, (x, y), (x + w, y + h), color, 2)
#             cv2.putText(img, f'{label, confidence_label}', (x-25, y + 75), font, 2, color, 2)
            
            cv2.rectangle(img_1, (xc-8, yc), (xc2-8, yc2), color, 2) #(yc*255/height)
            cv2.putText(img_1, 'V', (xc-8, yc2+16), font, 0.8, color, 2)
            
    
    img_1 = cv2.flip(img_1, 0)
    cv2.imshow('Single Channel Window', img_1)
    
    cv2.imshow("Image", img)

    if cv2.waitKey(1) & 0xFF == 27: # on press of Ecs break
        break

## Change in way of calculation of x coordinate of position
### Also Increased Number of Lanes

In [ ]:
# read video from file
cap = cv2.VideoCapture('trim16.mp4')

while True:
    read_ok, img = cap.read()
 
    height, width, channels = img.shape
    
    # Detecting objects
    blob = cv2.dnn.blobFromImage(img, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
    net.setInput(blob)
    outs = net.forward(output_layers)
 
    # Showing informations on the screen
    class_ids = []
    confidences = []
    boxes = []
    cars = []
    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.5:
                # Object detected
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                w = int(detection[2] * width)
                h = int(detection[3] * height)
                # Mapping into 2D
                yc = int(10*math.tan((1 - detection[1] - 0.5*detection[3]) *math.pi/2 + 0.83))
                # scaling factor* is not effective enough. so adding theta bias
                
                xc = int(200 + (yc+5)*(center_x*400/width + np.sign(center_x - width/2) *w*200/width -190)/240)
                
#                 xc = int((center_x*400/width - 190)*(yc+15)/280 +200) #int(detection[0] * yc)
                xc2 = xc + 16
                yc2 = yc + 16
                
                # Rectangle coordinates
                x = int(center_x - w / 2)
                y = int(center_y - h / 2)
                boxes.append([x, y, w, h])
                cars.append([xc, yc, xc2, yc2])
                confidences.append(float(confidence))
                class_ids.append(class_id)
 
    indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)
 
    font = cv2.FONT_HERSHEY_DUPLEX
    
    img_1 = np.zeros([650,400,3],dtype=np.uint8)
    img_1.fill(255)
    cv2.rectangle(img_1, (190, 0), (210, 800), (255,220,255), -1)
    cv2.rectangle(img_1, (210, 0), (230, 800), (255,255,220), -1)
    cv2.rectangle(img_1, (230, 0), (250, 800), (255,220,255), -1)
    cv2.rectangle(img_1, (250, 0), (270, 800), (255,255,220), -1)
    cv2.rectangle(img_1, (270, 0), (290, 800), (255,220,255), -1)
    
    cv2.rectangle(img_1, (130, 0), (150, 800), (255,255,220), -1)
    cv2.rectangle(img_1, (150, 0), (170, 800), (255,220,255), -1)
    cv2.rectangle(img_1, (170, 0), (190, 800), (255,255,220), -1)
    cv2.rectangle(img_1, (192, 0), (208, 16), (0,255,0), 2)
    cv2.putText(img_1, 'V', (192, 32), font, 0.8, (0,255,0), 2)
    
    for i in range(len(boxes)):
        if i in indexes:
            x, y, w, h = boxes[i]
            xc, yc, xc2, yc2 = cars[i]
            label = str(classes[class_ids[i]])
            confidence_label = int(confidences[i] * 100)
            color = colors[i]
            cv2.rectangle(img, (x, y), (x + w, y + h), color, 2)
#             cv2.putText(img, f'{label, confidence_label}', (x-25, y + 75), font, 2, color, 2)
            
            cv2.rectangle(img_1, (xc-8, yc), (xc2-8, yc2), color, 2) #(yc*255/height)
            cv2.putText(img_1, 'V', (xc-8, yc2+16), font, 0.8, color, 2)
            
    
    img_1 = cv2.flip(img_1, 0)
    cv2.imshow('Single Channel Window', img_1)
    
    cv2.imshow("Image", img)

    if cv2.waitKey(1) & 0xFF == 27: # on press of Ecs break
        break

# Skipping frames for faster output

In [7]:
# read video from file
cap = cv2.VideoCapture('trim16.mp4')
i = 0
while True:
    read_ok, img = cap.read()
 
    height, width, channels = img.shape
    
    i += 1
    
    if np.remainder(i, 10) == 0:
        # Detecting objects
        blob = cv2.dnn.blobFromImage(img, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
        net.setInput(blob)
        outs = net.forward(output_layers)

        # Showing informations on the screen
        class_ids = []
        confidences = []
        boxes = []
        cars = []
        for out in outs:
            for detection in out:
                scores = detection[5:]
                class_id = np.argmax(scores)
                confidence = scores[class_id]
                if confidence > 0.5:
                    # Object detected
                    center_x = int(detection[0] * width)
                    center_y = int(detection[1] * height)
                    w = int(detection[2] * width)
                    h = int(detection[3] * height)
                    # Mapping into 2D
                    yc = int(10*math.tan((1 - detection[1] - 0.5*detection[3]) *math.pi/2 + 0.83))
                    # scaling factor* is not effective enough. so adding theta bias

                    xc = int(200 + (yc+5)*(center_x*400/width + np.sign(center_x - width/2) *w*200/width -190)/240)

    #                 xc = int((center_x*400/width - 190)*(yc+15)/280 +200) #int(detection[0] * yc)
                    xc2 = xc + 16
                    yc2 = yc + 16

                    # Rectangle coordinates
                    x = int(center_x - w / 2)
                    y = int(center_y - h / 2)
                    boxes.append([x, y, w, h])
                    cars.append([xc, yc, xc2, yc2])
                    confidences.append(float(confidence))
                    class_ids.append(class_id)

        indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)

        font = cv2.FONT_HERSHEY_DUPLEX

        img_1 = np.zeros([650,400,3],dtype=np.uint8)
        img_1.fill(255)
        cv2.rectangle(img_1, (190, 0), (210, 800), (255,220,255), -1)
        cv2.rectangle(img_1, (210, 0), (230, 800), (255,255,220), -1)
        cv2.rectangle(img_1, (230, 0), (250, 800), (255,220,255), -1)
        cv2.rectangle(img_1, (250, 0), (270, 800), (255,255,220), -1)
        cv2.rectangle(img_1, (270, 0), (290, 800), (255,220,255), -1)
        
        cv2.rectangle(img_1, (130, 0), (150, 800), (255,255,220), -1)
        cv2.rectangle(img_1, (150, 0), (170, 800), (255,220,255), -1)
        cv2.rectangle(img_1, (170, 0), (190, 800), (255,255,220), -1)
        cv2.rectangle(img_1, (192, 0), (208, 16), (0,255,0), 2)
        cv2.putText(img_1, 'V', (192, 32), font, 0.8, (0,255,0), 2)

        for i in range(len(boxes)):
            if i in indexes:
                x, y, w, h = boxes[i]
                xc, yc, xc2, yc2 = cars[i]
                label = str(classes[class_ids[i]])
                confidence_label = int(confidences[i] * 100)
                color = colors[i]
                cv2.rectangle(img, (x, y), (x + w, y + h), color, 2)
    #             cv2.putText(img, f'{label, confidence_label}', (x-25, y + 75), font, 2, color, 2)

                cv2.rectangle(img_1, (xc-8, yc), (xc2-8, yc2), color, 2) #(yc*255/height)
                cv2.putText(img_1, 'V', (xc-8, yc2+16), font, 0.8, color, 2)


        img_1 = cv2.flip(img_1, 0)
        cv2.imshow('Single Channel Window', img_1)

        cv2.imshow("Image", img)

    if cv2.waitKey(1) & 0xFF == 27: # on press of Ecs break
        break

IndexError: index 20 is out of bounds for axis 0 with size 20